In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls "/content/drive/My Drive/evodata"

X1.h5  Y1.h5


In [3]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [0]:
!cp "/content/drive/My Drive/evodata/Y1.h5" data/

In [0]:
!cp "/content/drive/My Drive/evodata/X1.h5" data/

In [4]:
!ls data

X1.h5  Y1.h5


In [0]:
!chmod 777 data/X1.h5

In [0]:
!chmod 777 data/Y1.h5

In [13]:
# import numpy as np
# np.__version__

!pip install numpy==1.15.4

     |████████████████████████████████| 13.9MB 3.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [25]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 4.2MB/s 


In [0]:
import pandas as pd
import numpy as np
from torch.autograd import Variable
from skmultilearn.problem_transform import LabelPowerset

In [0]:
go_terms_to_classify = ['go_9401', 'go_45454']

In [0]:
with pd.HDFStore('data/Y1.h5',  mode='r') as store:
    Y = store.select('Y')

with pd.HDFStore('data/X1.h5',  mode='r') as store:
    X = store.select('X')

In [5]:
X['InclusionBodiesProbPos'] = (X['InclusionBodiesProbability']>=0)*X['InclusionBodiesProbability']
X['InclusionBodiesProbNeg'] = (X['InclusionBodiesProbability']<0)*X['InclusionBodiesProbability']


def location_type_numeric(x1):
    if x1 == '0':
        return 1
    elif x1 == 'SignalP_gram_plus_S':
        return 2
    elif x1 == 'SignalP_gram_minus_S':
        return 3
    elif x1 == 'T4SEpre_bpbAac_T4SE':
        return 4
    elif x1 == 'hmmsearch_Tat':
        return 5
    elif x1 == 'T3_MM_T3SS':
        return 6
    elif x1 == 'SignalP_gram_plus_S':
        return 7
    elif x1 == 'hmmsearch_T6S':
        return 8
    elif x1 == 'hmmsearch_T1S':
        return 9
    else:
        raise Exception(x1)

X['loc_type_numeric'] = X['loc_type'].apply(location_type_numeric)


Y.sort_values('cluster', inplace=True)
X.sort_values('cluster_is', inplace=True)

Y.drop('cluster', inplace=True,axis=1)
X.drop(['cluster_is', 'InclusionBodiesProbability', 'loc_type' ], inplace=True,axis=1)



Y = Y[go_terms_to_classify] if go_terms_to_classify is not None else Y

NameError: ignored

In [0]:

from collections import Counter
from sklearn.datasets import make_classification
#from imblearn.under_sampling import RandomUnderSampler # doctest: +NORMALIZE_WHITESPACE
from imblearn.over_sampling import RandomOverSampler
rus = RandomOverSampler(random_state=42)


import torch.nn.functional as F
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)



#here
from skmultilearn.problem_transform import LabelPowerset
lp = LabelPowerset()
yt = lp.transform(y_train)
X_res, y_res = rus.fit_resample(X_train.values, yt)

y_res = lp.inverse_transform(y_res)
y_res = y_res.todense()
y_res =np.array(y_res)
#here

#X_res, y_res = rus.fit_resample(X_train.values, y_train[['go_9401', 'go_45454']].values)

#y_test = y_test[['go_9401', 'go_45454']]




In [0]:

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


class DatasetLoader(Dataset):

    def __init__(self, x_df, y_df):
        self.x_df = x_df
        self.y_df = y_df


    def __len__(self):
        return len(self.x_df)

    def __getitem__(self, idx):
        x = self.x_df[idx]
        #y = self.y_df.iloc[idx]['go_9401']
        y = self.y_df[idx]

        return x.reshape(-1), y.reshape(-1)

        # return np.asarray(x.tolist()).reshape(-1), np.asarray([y]).reshape(-1)


train_dataset_loader = DatasetLoader(X_res, y_res)
train_loader = DataLoader(train_dataset_loader, batch_size=100, shuffle=True, num_workers=0)


test_dataset_loader = DatasetLoader(X_test.values, y_test.values)
test_loader = DataLoader(test_dataset_loader, batch_size=100, shuffle=True, num_workers=0)


class GoTermNN(nn.Module):
    def __init__(self, num_inputs=62, num_classes=1, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 50),
            nn.ReLU(),

            nn.Linear(50, 20),
            nn.ReLU(True),

            nn.Linear(20, 2),
            # nn.Dropout(dropout_prob),
            # nn.Softmax(1)
        )


    def forward(self, x):
        return self.pipe(x)


from sklearn.metrics import roc_auc_score

def train(model, device, train_loader, optimizer, epoch, writer, criterion):
    model.train()
    running_loss = 0.0
    running_correct = 0.0
    calc_count = 0.0
    labels_list = []
    scores_list = []
    loss1 = 0.1
    for x, (images, labels) in enumerate(train_loader):
        images = images.float().to(device)
        labels = labels.float().to(device)
        optimizer.zero_grad()

        images = Variable(images)
        labels = Variable(labels)

        output = model(images)
        calc_count += output.data.shape[0]
        _, pred = torch.max(output.data, 1)
        loss = criterion(output, labels)

        out1 = F.sigmoid(output)

        #labels_list.extend(batch['label'].tolist())

        labels_list.extend(labels.numpy().tolist())
        #scores_list.extend(out1.data.view(-1).tolist())
        scores_list.extend(out1.data.tolist())

        #running_correct += torch.sum(output.data == labels.data).cpu().numpy().item()
        current_loss = loss.item()
        running_loss += current_loss

        loss1 += current_loss

        if x > 0 and  x %10000 == 0:

            auc = roc_auc_score(np.asarray(labels_list, dtype='float32'), np.asarray(scores_list, dtype='float32'))
            print('auc {0}'.format(auc))
            print('loss1 {0}'.format(loss1))

            labels_list = []
            scores_list = []
            
            loss1 = 0.0


        #writer.add_scalar('train_loss', loss.item())

        loss.backward()
        optimizer.step()

    accuracy = round(running_correct / calc_count, 4)
    #writer.add_scalar('train_accuracy', accuracy)
    return round(running_loss / calc_count, 4), accuracy


def test(model, device, test_loader, epoch, writer, criterion):
    model.eval()
    running_correct = 0.0
    running_loss = 0.0
    calc_count = 0.0
    labels_list = []
    scores_list = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.float().to(device)
            labels = labels.float().to(device)

            images = Variable(images)
            labels = Variable(labels)
            outputs = model(images)


            #_, pred = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            calc_count += outputs.data.shape[0]
            #running_correct += torch.sum(pred == labels.data).cpu().numpy().item()
            running_loss += loss.item()

            out1 = F.sigmoid(outputs)
            labels_list.extend(labels.numpy().tolist())
            scores_list.extend(out1.data.tolist())


            #scores_list.extend(outputs.data.view(-1).tolist())



            #writer.add_scalar('train_loss', loss.item())

    auc = roc_auc_score(np.asarray(labels_list, dtype='float32'), np.asarray(scores_list, dtype='float32'))

    print('test auc {0}'.format(auc))
    accuracy = round(running_correct / calc_count, 4)
    #writer.add_scalar('train_accuracy', accuracy)

    return round(running_loss / calc_count, 4), accuracy

In [0]:


from datetime import datetime

use_cuda = torch.cuda.is_available()

torch.manual_seed(999)

if use_cuda:
    torch.cuda.manual_seed(999)
    torch.cuda.manual_seed_all(999)

print('use_cuda = {}'.format(use_cuda))
device = torch.device("cuda" if use_cuda else "cpu")
#criterion = nn.CrossEntropyLoss().to(device)

criterion = torch.nn.BCEWithLogitsLoss().to(device)

torch.manual_seed(999)
now = datetime.now()

# train_loader = get_loader(train=True, batch_size=args.batch_size, use_cuda=use_cuda)
# test_loader = get_loader(train=False, batch_size=args.test_batch_size, use_cuda=use_cuda)
model = GoTermNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    print('epoch={}'.format(epoch))
    train_loss, train_correct = train(model, device, train_loader, optimizer, epoch, None, criterion)
    print('train {0} - {1}'.format(train_loss, train_correct))
    test_loss, test_correct = test(model, device, test_loader, epoch, None, criterion)

    print('test {0} - {1}'.format(test_loss, test_correct))



print(1)



use_cuda = False
epoch=0


/home/ubuntu/.local/lib/python3.6/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


auc 0.6949790474623374
loss1 9246.46248126626


/home/ubuntu/.local/lib/python3.6/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


auc 0.8472225743082902
loss1 4511.025020211935


...

auc 0.9812026788639174
loss1 1576.5264971926808


/home/ubuntu/.local/lib/python3.6/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


auc 0.9813793624617043
loss1 1569.225318595767


/home/ubuntu/.local/lib/python3.6/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train 0.0016 - 0.0
test auc 0.9717963908676974
test 0.0022 - 0.0
1


use_cuda = False
